# `QuantumOps` performance intro

This notebook introduces some of `QuantumOps` with an emphasis on performance aspects.

Tell `PyCall` to use our virtual Python environment

In [1]:
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")

# import all default symbols for interactive demo
using QuantumOps
using BenchmarkTools
import LinearAlgebra
import SparseArrays

# We also import I, X, Y, Z for convenience
using QuantumOps.Paulis: I, X, Y, Z

[ Info: Precompiling QuantumOps [d0cc4389-ef80-471f-8a8c-dd305d114ada]


### Pauli

These, `I, X, Y, Z`, are bound to instances of the `Pauli` type, representing single-qubit operators.

In [2]:
(I, X, Y, Z) == Pauli.((0, 1, 2, 3)) ## The '.' broadcasts over the following elements.

true

Julia has a large number of standard interfaces and functions for numeric and algebraic types. I follow these when possible. For example, `Matrix` is used to construct a dense, heap-allocated matrix from an object. So I defined a method for it.

In [3]:
print(Matrix.(Pauli.(0:3)))

Matrix[[1.0 0.0; 0.0 1.0], [0.0 1.0; 1.0 0.0], ComplexF64[0.0 + 0.0im 0.0 - 1.0im; 0.0 + 1.0im 0.0 + 0.0im], [1.0 0.0; 0.0 -1.0]]

`Pauli` is in this type hierarchy.

In [4]:
Pauli <: AbstractPauli

true

Only a very small amount of code depends on the internals of `Pauli`. Almost everything is coded against `AbstractPauli`. The developer almost never encounters the implementation of `Pauli`, and the user never does.
But, if you want, you can see it this way.

In [5]:
dump(X)

QuantumOps.Paulis.Pauli
  hi: Bool false
  lo: Bool true


The notation `X[i, j]` calls `getindex(X, i, j)` which, for `AbstractPauli`, looks up the elements in stack allocated arrays. This is faster than indexing into a heap allocated (that is, every-day, dynamic) array:

In [6]:
m = rand(2, 2)
@btime $m[1, 1];

  1.816 ns (0 allocations: 0 bytes)


In [7]:
@btime X[1, 1];  ## This includes time to look up what matrix corresponds to `X`

  1.397 ns (0 allocations: 0 bytes)


Some linear algebra has been implemented.

In [8]:
Y * rand(2, 2)

2×2 Matrix{ComplexF64}:
 0.0-0.87131im   0.0-0.699078im
 0.0+0.204809im  0.0+0.342418im

These are more efficient than for heap-allocated arrays (except for BLAS), but still not the best, since we know the answers. It would be a good idea to hard code the result for `sparse` and return an copy. As an example I did implement methods for a few functions, for example, `ishermitian`, and `eigvals`:

In [9]:
@btime LinearAlgebra.eigvals(Z)

  19.824 ns (1 allocation: 80 bytes)


2-element Vector{Float64}:
 -1.0
  1.0

`20`ns is the time required to copy the array of eigenvalues.

### `PauliTerm`

`PauliTerm` represents a tensor product of Pauli operators (or a single one) and keeps track of a coefficient, including a phase.

### Compare `PauliTerm` with qiskit

In [10]:
using PyCall
qi = pyimport("qiskit.quantum_info");

Here, we compare multiplication of two Pauli strings with both libraries. We see how the time scales with string length.

In [11]:
function get_julia_python_terms(n_qubits)
    xj = PauliTerm(rand(Pauli, n_qubits))
    yj = PauliTerm(rand(Pauli, n_qubits))
    xp = qi.random_pauli(n_qubits)
    yp = qi.random_pauli(n_qubits)
    return (xj, yj, xp, yp)
end

n_qubits = 10
(xj, yj, xp, yp) = get_julia_python_terms(n_qubits)

# `QuantumOps`
@btime $xj * $yj

  51.781 ns (1 allocation: 80 bytes)


10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Complex{Int64}}:
YXYIZXZXXI * (0 + 1im)

In [12]:
# qiskit
@btime $xp.compose($yp)   ## @btime gives same times as %timeit in python cli

  16.692 μs (4 allocations: 256 bytes)


PyObject Pauli('-YYIYZXXZYZ')

#### For mulitplying 10-qubit Pauli strings, `QuantumOps` is about 300 times faster than qiskit.

In [13]:
julia_time = @belapsed $xj * $xj
qiskit_time = @belapsed $xp.compose($yp)

qiskit_time / julia_time

305.24450441333556

Asymptotically, qiskit is about three times faster than `QuantumOps`. But, it takes a while to get there. For 1000-qubit strings `QuantumOps` is still eight times faster. I have some ideas regarding why python is faster than Julia here, but I am not at all sure. Also, there is a big, ~12 micro-s constant term in the python times. It might be worth trying to reduce this.

In [14]:
n_qubits = 1000
(xj, yj, xp, yp) = get_julia_python_terms(n_qubits)

julia_time = @belapsed $xj * $xj
qiskit_time = @belapsed $xp.compose($yp)

qiskit_time / julia_time

9.475578128376918

Here are $10^4$ qubits. Julia is still faster, but they are comparable.

In [15]:
n_qubits = 10^4
(xj, yj, xp, yp) = get_julia_python_terms(n_qubits)

julia_time = @belapsed $xj * $xj
qiskit_time = @belapsed $xp.compose($yp)

qiskit_time / julia_time

1.9043244563889565

### `PauliSum`

A `PauliSum` represents a sum of `PauliTerm`s, sorted in a canonical order.

In [16]:
n_qubits = 10
n_terms = 10
ps = PauliSum(rand(Pauli, (n_terms, n_qubits)), randn(n_terms))

10x10 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{Float64}}:
IXYXZZXXZZ * 1.1154842217222714
IZIYYYYZYZ * -1.1771162811334117
XXXXXZYXIY * 0.06142472936955288
XYIXXZXYXX * -0.26292292075441276
YIYZZZXXZI * -0.3195756045453568
YXXIXZIXXI * -0.31824504534858344
ZIXZYZXIIY * -0.9806361102356757
ZIZZYXIZXI * -0.13814138878378268
ZYYIZZYYYZ * -0.7960146056180916
ZZIZYIIXIX * 0.5868030228158364

In [17]:
x = ps[5]
(x, typeof(x))

(10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Float64}:
YIYZZZXXZI * -0.3195756045453568, QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Float64})

`add!` adds a `PauliTerm` in place. It does a sorted search to find the correct location.

In [18]:
add!(ps, -x)  ## add the additive inverse of a term

9x10 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{Float64}}:
IXYXZZXXZZ * 1.1154842217222714
IZIYYYYZYZ * -1.1771162811334117
XXXXXZYXIY * 0.06142472936955288
XYIXXZXYXX * -0.26292292075441276
YXXIXZIXXI * -0.31824504534858344
ZIXZYZXIIY * -0.9806361102356757
ZIZZYXIZXI * -0.13814138878378268
ZYYIZZYYYZ * -0.7960146056180916
ZZIZYIIXIX * 0.5868030228158364

The length of the sum is now 9 rather than 10.

In [19]:
length(ps)

9

In [20]:
n_qubits = 10
n_terms = 10
ps = PauliSum(rand(Pauli, (n_terms, n_qubits)), randn(n_terms));
size(ps)

(10, 10)

In [21]:
x = copy(ps[1])

10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Float64}:
IIYYXXIXZZ * -1.2937054189347024

In [22]:
@btime add!($ps, $x);

  60.801 ns (0 allocations: 0 bytes)


That seems a bit slow.

#### Pauli decomposition

Construct the Pauli decomposition of a matrix.

In [23]:
m = rand(4, 4)
s = PauliSum(m)

16x2 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{ComplexF64}}:
II * (0.5272758846044263 + 0.0im)
IX * (0.5346218917798263 + 0.0im)
IY * (0.0 - 0.1369339286124308im)
IZ * (-0.15501410250040235 + 0.0im)
XI * (0.5709063597713999 + 0.0im)
XX * (0.6772212134926611 + 0.0im)
XY * (0.0 - 0.01200854526006953im)
XZ * (-0.18029004072656765 + 0.0im)
YI * (0.0 - 0.17420586958866563im)
YX * (0.0 + 0.04319718078624407im)
YY * (0.038778729260992933 - 0.0im)
YZ * (0.0 + 0.0400463390189206im)
ZI * (-0.008888183525475019 + 0.0im)
ZX * (-0.07321659932152849 + 0.0im)
ZY * (0.0 - 0.009382225369305747im)
ZZ * (0.12780088347609728 + 0.0im)

Check that the decomposition is correct.

In [24]:
m ≈ Matrix(s)

true

Doing this decomposition is exponentially expensive. Here we compare the performance of Qiskit QI vs. QuantumOps.

In [25]:
n = 7
m = rand(2^n, 2^n);
julia_time = @belapsed PauliSum($m)

0.081796948

In [26]:
qi_op = qi.Operator(m)
qi_time = @elapsed qi.SparsePauliOp.from_operator(qi_op)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall ~/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


8.563799748

Ratio of times to do Pauli decomposition for random 7-qubit matrix

In [27]:
qi_time / julia_time

104.69583471500673

### Parametric types and composability

#### Z4Group

I implemented a type `Z4Group` that represents `(i, -1, -i, 1)`. This can be used to represent the Pauli group. The type `Z4Group` becomes part of the type of the term, which aids the compiler in devirtualizing and inlining.

In [28]:
t = PauliTerm(:XXY, Z4Group(im))
(t, typeof(t))

(+i XXY, QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, QuantumOps.Z4Groups.Z4Group})

In [29]:
v = PauliTerm(:ZXZ, Z4Group(1))

+1 ZXZ

In [30]:
t * v

+i YIX

#### Z4Group0

More interesting is `Z4Group0` which is `Z4Group` augmented by another `Bool` representing zero. This can represent `(0, im, -im, 1, -1)`. It supports multiplication of elements, but is only closed under addition where at least one operand is `0`. It will error if you don't respect this. This quasi-algebra is enough to represent and compute kronecker products of Pauli matrices. The structure is this

In [31]:
dump(Z4Group0(1))

QuantumOps.Z4Group0s.Z4Group0
  z4: QuantumOps.Z4Groups.Z4Group
    imag: Bool false
    minus: Bool false
  zero: Bool true


Note that this is a nested composite type. Nontheless an array of these is packed, with each element taking three bytes. Here is a packed two-dimensional array of `Z4Group0`.

In [32]:
a = rand(Z4Group0, (3,5))
a

3×5 Matrix{QuantumOps.Z4Group0s.Z4Group0}:
 -1  -i  -i  0   -i
 -i  0   +i  +1  -i
 +1  +i  +i  +i  0

In [33]:
sizeof(a)  ## (3 x 5) x 3 bytes

45

We see that computation with `Z4Group0` can be as fast as or faster than `Complex{Int}`.

In [34]:
a = rand(Z4Group0, 10^5);
@btime reduce(*, a)

  64.186 μs (1 allocation: 16 bytes)


0

In [35]:
anum = [convert(Complex{Int}, x) for x in a];
@btime reduce(*, anum)

  132.563 μs (1 allocation: 32 bytes)


0 + 0im

I use `Z4Group0` in Kronecker products.

In [36]:
kron([Z4Group0.(m) for m in Matrix.([X, Y, Z])]...)

8×8 Matrix{QuantumOps.Z4Group0s.Z4Group0}:
 0   0   0   0   0   0   -i  0
 0   0   0   0   0   0   0   +i
 0   0   0   0   +i  0   0   0
 0   0   0   0   0   -i  0   0
 0   0   -i  0   0   0   0   0
 0   0   0   +i  0   0   0   0
 +i  0   0   0   0   0   0   0
 0   -i  0   0   0   0   0   0

In [37]:
operators = rand(Pauli, 4)
print(operators)

IYIZ

In [38]:
mats = Matrix.(operators)
z40mats = [Z4Group0.(m) for m in mats];

size(kron(mats...))

(16, 16)

In [39]:
@btime kron($mats...);

  708.108 ns (3 allocations: 5.52 KiB)


In [40]:
@btime kron($z40mats...);

  675.150 ns (3 allocations: 1.12 KiB)


Here, the time to do the calcuations with usual 16-byte complex numbers is the same. But, when converting a `PauliSum` to a matrix I use `ThreadsX.sum` over the terms, which is a dropin replacement for `sum` that does intelligent threading. When I use `Z4Group0` I get a significant improvement in performance, perhaps because of fewer cache misses.

### sympy

In [41]:
@pyimport sympy
(x, t) = sympy.symbols("x t")

(PyObject x, PyObject t)

We use a symbolic coefficient

In [42]:
term = PauliTerm("XXYZ", x + t)

4-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, PyCall.PyObject}:
XXYZ * (PyObject t + x)

In [43]:
term^3

4-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, PyCall.PyObject}:
XXYZ * (PyObject (t + x)**3)

The type of coefficient is encoded in the type of the `PauliTerm`.

In [44]:
typeof(term)

PauliTerm{Vector{Pauli}, PyObject} (alias for QuantumOps.OpTerm{QuantumOps.Paulis.Pauli, Array{QuantumOps.Paulis.Pauli, 1}, PyCall.PyObject})

#### `Symbolics`

This is another symbolic libarary.

The following is disabled because of errors due to changes in packages.

using Symbolics
#----------------------------------------------------------------------------

@variables a b c;
#----------------------------------------------------------------------------

# Create a `PauliSum` with symbolic coefficients

term1 = PauliTerm("XZ", a + b)
term2 = PauliTerm("ZX", b + c)

psum = term1^3 + term2
#----------------------------------------------------------------------------

# We convert the `PauliSum` with symbolic coefficients to a `Matrix`.
# No additional code is necessary to support this feature.

symmat = Matrix(psum)
#----------------------------------------------------------------------------

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*